In [18]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import xlsxwriter

writer = pd.ExcelWriter('C:\\Rebate\\202202 All_Rebate Feb (1-27).xlsx', engine='xlsxwriter')

## Getting the eods
eod = pd.read_csv("C:\\Users\\oazed\\Downloads\\2022-02-28 031832 - Hub Symbol EOD Price History - default.csv")
eod = eod[['Timestamp','Symbol', 'Bid Price']]
eod = eod.rename(columns={'Timestamp':'date'})
eod["date"] = pd.to_datetime(eod["date"])
eod['date'] = eod['date'].dt.date
eod = eod.rename({'Symbol':'instrument'},axis=1)
### EOD dictionary for translate
eods_dict= {
    'US500' : 'SPM/USD',
    'NAS100': 'NDM/USD',
    'US30' : 'DJI/USD',
    'AUS200': 'AXI/AUD',
    'EUSTX50':'STX/EUR',
    'HK50': 'HSI/HKD',
    'JPN225': 'JPU/JPY',
    'UK100' : 'UKI/GBP',
    'GER30': 'GEM/EUR',
    'CN50' : 'CNI/USD',
    'FRA40':'FRI/EUR',
}
 
#### Function that translate
def banao(x):
    ins = x['instrument']
    if ins in eods_dict.keys():
        return eods_dict[ins]
    else:
        return ins
    
vals = eod.apply(banao, axis=1)
eod['instrument'] = vals

df = pd.read_csv('C:\\Users\\oazed\\Downloads\\2022-02-28 031732 - Hub Trades List - default.csv')
df = df[df['Filled Volume'] != 0]
df = df[df['Request Source']  != 'Swap']
df = df[df['Request Source'] != 'Adjustment']

# DST and EST timezone adjustments
def timezone_adjustments(date1):
    date1 = date1.split('.', 1) # Remove miliseconds
    date1 = date1[0]
    date1 = datetime.strptime(date1, '%Y-%m-%d %H:%M:%S')
    if datetime(2021, 11, 7, 2, 0, 0) <= date1 <= datetime(2022, 3, 14, 2, 0, 0):
        date2 = date1 + timedelta(hours = 2)
    else:
        date2 = date1 + timedelta(hours = 3)    
    return date2
df['Executed Time'] = df['Executed Time'].apply(lambda x: timezone_adjustments(x))
# p2['Executed Time'] = pd.to_datetime(p2['Executed Time'])
# p3 = p2[p2['Executed Time'].dt.weekday < 5]
# df = pd.concat([df,p3])
# df['Executed Time'] = pd.to_datetime(df['Executed Time'])

######## df['Executed Time'] = df['Executed Time']+pd.offsets.Hour(3)
df = df.rename(columns={'Core Symbol':'instrument', 'Executed Time':'date'})

df = df[['date','instrument','Request Source','Taker Name','Taker Login','Taker Group','Filled Volume','Maker Stream Name', 'Maker Executed Price','Volume in USD','Term CY Conv Ratio' ]]

cryptos =['BTC/USD', 'BCH/USD', 'EOS/USD','ETH/USD','LTC/USD', 'XRP/USD','LNK/USD','UST/USD']
cfds = [ 'US500','NAS100','US30' ,'AUS200','EUSTX50','HK50','JPN225','UK100' ,'GER30','CN50' ,'FRA40']
metals = ['XAU/USD','XAG/USD']
def custom_instrument_oz(x):
    vals = x['instrument']
    lis = ["CRYPTOS", "CFDS","FX","STOCKS", "METAL"]
    if vals in cryptos:
        return lis[0]
    if vals in cfds:
        return lis[1]
    if vals in metals:
        return lis[4]
    if "." in vals:
        return lis[3]
    else:
        return lis[2]
df["category"] = df.apply(custom_instrument_oz,axis=1)

def abar_banao(x):
    ins = x['instrument']
    if ins in eods_dict.keys():
        return eods_dict[ins]
    else:
        return ins
    
df_vals = df.apply(abar_banao, axis=1)
df['instrument'] = df_vals


other_dict= {
    'SPM/USD' : 'SPM/USD',
    'NDM/USD': 'NDM/USD',
    'DJI/USD' : 'DJI/USD',
    'AXI/AUD': 'AXI/AUD',
    'STX/EUR':'STX/EUR',
    'HSI/HKD': 'HSI/HKD',
    'JPU/JPY': 'JPU/JPY',
    'UKI/GB' : 'UKI/GBP',
    'GEM/EUR': 'GEM/EUR',
    'CNI/USD' : 'CNI/USD',
    'FRI/EUR':'FRI/EUR',
    'XBRUSD':'XBRUSD',
    'XTIUSD':'XTIUSD',
    'WTIUSD':'WTIUSD'
}

#### Function to add eod start here 
conv = dict()

def findUSD(s):
    date = s['date']
    ins = s['instrument']
    rate = s['Bid Price']
    
    if '/' not in ins:
        if ins.endswith('USD'):
            conv[date][ins] = rate
    x = ins.split('/')

    if date not in conv:
        conv[date] = dict()
        conv[date]['USD/USD'] = 1.0
    
    if len(x) != 2:
        return
        
    if x[1] == 'USD':
        conv[date][ins] = rate
        
    if x[0] == 'USD':
        ins = x[1] + '/' + x[0]
        conv[date][ins] = 1/rate

def applyUSD(s):
    date = s['date']
    ins = s['instrument']
    rate = s['Bid Price']
    
    x = ins.split('/')
    
    if len(x) != 2:
        for kwd in other_dict.keys():
            if kwd in ins:
                if other_dict[kwd] in conv[date]:
                    return conv[date][other_dict[kwd]]
     
    if 'USD' not in x:
        if x[0] == 'AXI':
            return conv[date]['AUD/USD'] * rate
        if x[0] == 'UKI':
            return conv[date]['GBP/USD'] * rate
        if x[0] == 'GEM':
            return  conv[date]['EUR/USD'] * rate
        if x[0] == 'HSI':
            return conv[date]['HKD/USD'] * rate
        if x[0] == 'JPU':
            return conv[date]['JPY/USD'] * rate
        if x[0] == 'FRI':
            return conv[date]['EUR/USD'] * rate
        
        
    if 'USD' in x:
        if x[0] == 'USD':
            return 1/rate
        elif x[1] == 'USD':
            return rate
    
    ins0 = x[0]+'/'+'USD'
#     ins1 = x[1]+'/'+'USD'
    
    if ins0 in conv[date]:
        return conv[date][ins0]
    
#     if ins1 in conv[date]:
#         return rate * conv[date][ins1]
        
eod.apply(findUSD, axis=1)

eod['EoD_IN_USD'] = eod.apply(applyUSD, axis=1)

eod.set_index(['date','instrument'], inplace=True)
df['date'] = df['date'].dt.date
df.set_index(['date','instrument'], inplace=True)

omni = df.join(eod, how='left')
omni= omni.reset_index()
omni['Vols_in_USD'] = omni['Filled Volume']* omni['EoD_IN_USD']

#### Handling USD trades 
def eod_usd(x):
    val1 = x['Vols_in_USD']
    val2 = x['Filled Volume']
    val3 = x['instrument']
    ins = val3.split('/')
    if ins[0] == 'USD':
        return val2
    else:
        return val1
    
omni['Vols_in_USD'] = omni.apply(eod_usd,axis=1)

clients = {
    'APT001':'House',
    'APT701':'FooFx',
    'APT702':'Azure',
    'APT703':'Strathos',
    'APT704': 'Realms',
    'JDR001':'JDR',
    'JDR002':'JDR',
    'JDRSecurities':'JDR',
    'Warehouse':'Internal',
    'NeptuneSecurities': 'Neptune',
    'MT4_AlphaTest':'Internal',
    'MT4_APT703':'Strathos',
    'MT4_APT704':'Realms',
    'MT4_APT701':'FooFx',
    'MT4_APT702':'Azure'
}
def client_name(x):
    name = x['Taker Name']
    if name in clients:
        return clients[name]
    else:
        return name

new = omni.apply(client_name,axis=1)

omni['Taker Name'] = new

omni = omni.rename(columns={'Taker Name':'Client Name'})

foofx = omni[omni['Client Name']== 'FooFx'].reset_index()
azure = omni[omni['Client Name']== 'Azure'].reset_index()
stra = omni[omni['Client Name']== 'Strathos'].reset_index()
real = omni[omni['Client Name']== 'Realms'].reset_index()
nep = omni[omni['Client Name']== 'Neptune'].reset_index()

foofx = foofx.drop(columns='index')
azure = azure.drop(columns='index')
stra = stra.drop(columns='index')
real = real.drop(columns='index')
nep = nep.drop(columns='index')

In [21]:


foofx = foofx.groupby(["Client Name","category"]).agg({"Vols_in_USD":"sum"}).reset_index()
azure = azure.groupby(["Client Name","category"]).agg({"Vols_in_USD":"sum"}).reset_index()
real = real.groupby(["Client Name","category"]).agg({"Vols_in_USD":"sum"}).reset_index()
nep = nep.groupby(["Client Name","category"]).agg({"Vols_in_USD":"sum"}).reset_index()


foofx['Rebate'] = foofx.Vols_in_USD * 2
## Azure Condition for Rebate

if azure.Vols_in_USD.sum()>5e9:
    azure['Rebate'] = azure.Vols_in_USD * 2
else:
    azure['Rebate'] = 0 

real['Rebate'] = real.Vols_in_USD * 2
nep['Rebate'] = nep.Vols_in_USD * 2


foofx.Vols_in_USD = foofx.Vols_in_USD/1e6
azure.Vols_in_USD = azure.Vols_in_USD/1e6
# stra.Vols_in_USD = stra.Vols_in_USD/1e6
real.Vols_in_USD = real.Vols_in_USD/1e6
nep.Vols_in_USD = nep.Vols_in_USD/1e6


all_df = pd.concat([foofx,azure, real,nep])

def all_reb(x):
    cat = x['category']
    vol = x['Vols_in_USD']
    reb = x['Rebate']
    if 'CRYPTOS' in cat:
        return 100 * vol
    else:
        return reb
        
all_df['Rebate'] = all_df.apply(all_reb,axis=1)

all_df.Vols_in_USD = all_df.Vols_in_USD*1e6


def final(x):
    name = x['Client Name']
    if name == 'FooFx':
        return 'Lawrence'
    if name == 'Azure':
        return name
    if name == 'Realms':
        return 'Romano Liew'
    if name == 'Red Summit':
        return 'Shan'
    if name == 'Neptune':
        return 'Wei Wenhan'
    


all_df['Client'] = all_df.apply(final,axis=1)

all_df = all_df.reset_index()
all_df= all_df.drop('index', 1)
all_df = all_df.rename(columns={'Client Name':'Broker-Dealer', 'Vols_in_USD':'Volumes($M)'})
all_df = all_df[['Client','Broker-Dealer', 'category', 'Volumes($M)', 'Rebate']]
all_df['id'] = all_df.groupby(['Client','Broker-Dealer']).ngroup()
all_df= all_df.sort_values('id')
all_df = all_df[['id','Client','Broker-Dealer', 'category', 'Volumes($M)', 'Rebate']]
all_df.to_excel(writer, sheet_name='Sheet1', index=False)    


In [25]:
def strathos(x):
    taker = x['Taker Group']
    taker_list =['SC-I-AT0-7-USD']
    if taker in taker_list:
        return taker
    
stra['Taker Group'] = stra.apply(strathos,axis=1)
stra = stra.dropna()

stra = stra.groupby(["Client Name","Taker Login","category"]).agg({"Vols_in_USD":"sum"}).reset_index()
stra.Vols_in_USD = stra.Vols_in_USD / 1e6


def strathos_reb(x):
    cat = x['category']
    vol = x['Vols_in_USD']
    if 'CRYPTOS' in cat:
        return 100 * vol
    else:
        return  2.5 * vol
        
stra['Rebate'] = stra.apply(strathos_reb,axis=1)

stra = stra.groupby(["Client Name","Taker Login","category"]).agg({"Vols_in_USD":"sum", "Rebate":"sum"}).reset_index()

stra['Client'] = 'SHAN'

stra.Vols_in_USD = stra.Vols_in_USD * 1e6

stra = stra.rename(columns={'Client Name':'Broker-Dealer', 'Vols_in_USD':'Volumes($M)','Taker Login':'ID'})
stra =  stra[['Client','Broker-Dealer','ID','category', 'Volumes($M)', 'Rebate']]
stra.to_excel(writer, sheet_name='Sheet2', index=False)    
writer.save()


In [26]:
stra

,Client,Broker-Dealer,ID,category,Volumes($M),Rebate
0,SHAN,Strathos,800000471,CRYPTOS,1.252732e+04,1.252732
1,SHAN,Strathos,800000471,METAL,4.142208e+04,0.103555
2,SHAN,Strathos,800000472,CRYPTOS,1.543718e+04,1.543718
3,SHAN,Strathos,800000472,METAL,2.701440e+04,0.067536
4,SHAN,Strathos,800000473,CFDS,1.951675e+06,4.879189
5,SHAN,Strathos,800000473,CRYPTOS,1.083891e+06,108.389096
6,SHAN,Strathos,800000473,FX,2.695067e+04,0.067377
7,SHAN,Strathos,800000473,METAL,8.273214e+06,20.683036
